# Ejemplo de ETL: Limpieza de Nulos y Outliers

Aquí realizaremos un flujo ETL típico:
1. Carga de datos del Padrón Electoral.
2. Detección y limpieza de valores nulos.
3. Detección y filtrado de outliers (ejemplo: edades fuera de rango lógico).
4. Transformación de columnas y guardado.


In [0]:
from pyspark.sql.functions import col, when

from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType

padron_schema  = StructType(fields=[
    StructField("CEDULA",IntegerType(),True), 
    StructField("CODELEC",IntegerType(),True),
    StructField("RELLENO",StringType(),True),
    StructField("FECHACADUC",DateType(),True),
    StructField("JUNTA",IntegerType(),True),
    StructField("NOMBRE",StringType(),True),
    StructField("1_APELLIDO",StringType(),True),
    StructField("2_APELLIDO",StringType(),True),
])

df = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(padron_schema)\
    .csv("dbfs:/FileStore/tables/PADRON_COMPLETO.csv")



In [0]:
# 1. Inspección inicial
df.printSchema()



root
 |-- CEDULA: integer (nullable = true)
 |-- CODELEC: integer (nullable = true)
 |-- RELLENO: string (nullable = true)
 |-- FECHACADUC: date (nullable = true)
 |-- JUNTA: integer (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- 1_APELLIDO: string (nullable = true)
 |-- 2_APELLIDO: string (nullable = true)



In [0]:
df.select([col for col in df.columns]).show(5)



+---------+-------+-------+----------+-----+--------------------+--------------------+--------------------+
|   CEDULA|CODELEC|RELLENO|FECHACADUC|JUNTA|              NOMBRE|          1_APELLIDO|          2_APELLIDO|
+---------+-------+-------+----------+-----+--------------------+--------------------+--------------------+
|101053316| 104015|       |2028-02-07|    0|LUCILA           ...|PORRAS           ...|AGUERO           ...|
|101086526| 101012|       |2028-02-07|    0|DINORA           ...|OBANDO           ...|GARCIA           ...|
|101141655| 103033|       |2030-02-04|    0|TRINIDAD         ...|VINDAS           ...|PEREZ            ...|
|101142031| 112007|       |2030-06-30|    0|INOCENCIA        ...|MEZA             ...|VEGA             ...|
|101164392| 119001|       |2030-03-06|    0|JOSE FRANCISCO   ...|DUARTE           ...|QUESADA          ...|
+---------+-------+-------+----------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [0]:
# 2. Detección de nulos
for c in df.columns:
    print(c, "->", df.filter(col(c).isNull()).count(), "nulos")



CEDULA -> 0 nulos
CODELEC -> 0 nulos
RELLENO -> 0 nulos
FECHACADUC -> 0 nulos
JUNTA -> 0 nulos
NOMBRE -> 0 nulos
1_APELLIDO -> 0 nulos
2_APELLIDO -> 0 nulos


In [0]:
# 3. Limpieza: Eliminar filas con cédula o nombre nulo
df_clean = df.dropna(subset=["CEDULA", "NOMBRE"])



In [0]:
# 4. Supongamos que tenemos una columna EDAD y filtramos outliers
# (Si no hay, simular)
from pyspark.sql.functions import rand
df_clean = df_clean.withColumn("EDAD", (rand()*90+10).cast("int"))  # Simular edad



In [0]:
# Filtrar edades <18 o >100
df_clean = df_clean.filter((col("EDAD") >= 18) & (col("EDAD") <= 100))



In [0]:
# 5. Transformación: agregar columna "mayor_de_edad"
df_clean = df_clean.withColumn("mayor_de_edad", when(col("EDAD") >= 18, True).otherwise(False))

df_clean.select("CEDULA", "NOMBRE", "EDAD", "mayor_de_edad").show(5)

# Guardar resultado
df_clean.write.mode("overwrite").csv("dbfs:/FileStore/tables/padron_limpio.csv", header=True)

+---------+--------------------+----+-------------+
|   CEDULA|              NOMBRE|EDAD|mayor_de_edad|
+---------+--------------------+----+-------------+
|101053316|LUCILA           ...|  95|         true|
|101086526|DINORA           ...|  70|         true|
|101141655|TRINIDAD         ...|  40|         true|
|101142031|INOCENCIA        ...|  96|         true|
|101164392|JOSE FRANCISCO   ...|  76|         true|
+---------+--------------------+----+-------------+
only showing top 5 rows

